In [12]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import pandas as pd

In [13]:
# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[0:-64], data[-64:]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/4))
	test = array(split(test, len(test)/4))
	return train, test

In [14]:
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

In [15]:
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))
    
def to_supervised(train, n_input, n_out=4):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

In [16]:
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 0, 50, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	# define model
	model = Sequential()
	model.add(LSTM(250, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(250, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

In [17]:
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

In [18]:
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

In [19]:
df=pd.read_csv("FB.csv")
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
419,9/11/2020,270.059998,271.390015,262.640015,266.609985,266.609985,18913900
420,9/14/2020,270.950012,276.640015,265.700012,266.149994,266.149994,24093800
421,9/15/2020,270.670013,274.519989,269.299988,272.420013,272.420013,18478500
422,9/16/2020,267.290000,272.440000,261.790000,263.520000,263.520000,29183400
423,9/17/2020,258.280000,261.500000,250.190000,254.820000,254.820000,31281441


In [20]:
dataset=df[["Close"]]
train, test = split_dataset(dataset.values)
n_input = 10
history=[x for x in test]
model=build_model(train, n_input)
yhat=forecast(model,history,n_input)
yhat

array([[252.74184],
       [252.74577],
       [252.35965],
       [252.20726]], dtype=float32)

In [21]:
forecast=pd.DataFrame(yhat).rename(columns={0:"Forecast"})

In [22]:
import datetime

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

workdays=workdays(datetime.datetime(2020, 9, 18),
               datetime.datetime(2020, 9, 23))
workdays=pd.DataFrame(workdays).rename(columns={0:"Date"})

In [23]:
Facebook=pd.concat([workdays,forecast],axis=1)
Facebook

,Date,Forecast
0,2020-09-18,252.741837
1,2020-09-21,252.745773
2,2020-09-22,252.359650
3,2020-09-23,252.207260


In [24]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
419,9/11/2020,270.059998,271.390015,262.640015,266.609985,266.609985,18913900
420,9/14/2020,270.950012,276.640015,265.700012,266.149994,266.149994,24093800
421,9/15/2020,270.670013,274.519989,269.299988,272.420013,272.420013,18478500
422,9/16/2020,267.290000,272.440000,261.790000,263.520000,263.520000,29183400
423,9/17/2020,258.280000,261.500000,250.190000,254.820000,254.820000,31281441
